In [1]:
!pip install dominate

In [170]:
import dominate
import dominate.util
tags = dominate.tags
div, h3, code, span, a = tags.div, tags.h3, tags.code, tags.span, tags.a
text, raw = dominate.util.text, dominate.util.raw

with div("this") as this:
    this.add(div("that", id="that"))
    div("that2", id="that")
    with tags.ul():
        [tags.li(n) for n in range(3)]


from pygments import highlight
from pygments.lexers import HtmlLexer
from pygments.formatters import HtmlFormatter
    
def highlight_html(html):
    return highlight(
        html,
        HtmlLexer(),
        HtmlFormatter(noclasses=True))

class ReprHTMLConf:
    print_html = False
    print_highlight = False
    
class ReprHTML(object):
    def __init__(self, obj):
        self.obj = obj
    
    def __getattr__(self, attr):
        if attr.startswith('_repr'):
            if attr in ReprHTML:
                return object.__getattribute__(self, attr)
        return self.obj.__getattr__(self.obj, attr)
    
    def _repr_html_(self, obj=None):
        if obj is None:
            obj = self.obj
        if hasattr(obj, '_repr_html_'):
            obj_html = obj._repr_html_()
        else:
            obj_html = obj
        doc = div([
            div([h3('HTML'),
                 text(obj_html, escape=False),
                 #raw(obj_html)
                ]),
            div([h3('HTML (source)'),
                code(
                text(obj_html, escape=True))]),
            div([h3('HTML (highlighted)'),
                text(
                    highlight_html(obj_html),
                    escape=False)])
        ])
        doc_html = doc.render() # XXX: XSS
        if ReprHTMLConf.print_html:
            if ReprHTMLConf.print_highlight:
                print(doc_html)
            else:
                print(obj_html)
        return doc_html

#ReprHTMLConf.print_html = True
ReprHTML(this.render())

In [189]:
_jsonldstr = """
    {"@context": {
         "schema": "http://schema.org/",
         "jupyter": "https://jupyter.org/ns/v4/#",
         "_base": "http://localhost:8000/ns/v1#"
     },
     "@type": [
       "jupyter:JupyterNotebook",
       "schema:ScholarlyArticle",
       "schema:DataCatalog"
     ],
     "@id": "http://localhost:8888/notebooks/nb/nbmeta-00-01__exploration.ipynb",
     "name": "Notebook Name",
     "author": [{
       "@type": "schema:Person",
       "givenName": "Wesley",
       "familyName": "Turner",
       "url": "https://westurner.org/"
     }],
     "dateCreated": "2017-01-30",
     "about": [
         {"url": ["https://en.wikipedia.org/wiki/JSONLD"] },
         {"url": "https://pypi.org/project/pipfile/", "name": "Pipfile and Pipfile.lock"}
     ]
}
"""
import json
import collections


from pygments import highlight
from pygments.lexers import JavascriptLexer
from pygments.formatters import HtmlFormatter
    
def highlight_jsonld(jsonldstr):
    return highlight(
        jsonldstr,
        JavascriptLexer(),
        HtmlFormatter(noclasses=True))

Meta(ReprHTML(
    highlight_jsonld(_jsonldstr)),
    thing=json.loads(_jsonldstr, object_hook=collections.OrderedDict))

<type 'unicode'>


Exception: 

In [220]:


from collections import OrderedDict
class Meta(object):
    def __init__(self, obj, meta=None, **kwargs):
        self.obj = obj
        if meta is None:
            meta = OrderedDict()
        meta.update(kwargs)  # TODO
        self.meta = meta
        
    def __getattr__(self, attr):
        if 1: # attr.startswith('_repr'):
            if hasattr(Meta, attr):
                return object.__getattribute__(self, attr)
        return self.obj.__getattribute__(attr)
    
    def meta_to_html(self):
        with div(typeof='schema:CreativeWork jupyter:JupyterNotebook') as doc:
            def metahtml(items, cur_node):
                with cur_node as doc:
                    with tags.ul() as ul_node:
                        for key, value in items:
                            with tags.li(property=key) as li_node:
                                li_node.add(a(key, href=key, property="rdf:Predicate"))
                                if isinstance(value, basestring):
                                    span(text(value), property=key)
                                elif hasattr(value, 'items'):
                                    metahtml(value.items(), li_node)
                                elif isinstance(value, Meta):
                                    metahtml(value.obj, li_node)  # TODO: value.obj ->
                                elif hasattr(value, '__iter__'):
                                #elif isinstance(value, (list, tuple)):
                                    with tags.ul() as _ul_node:
                                        with tags.li() as _li_node:
                                            # TODO: BUG: XXX
                                            for _value in value:
                                                _items = OrderedDict(
                                                    (__value, __value) for __value in _value).items()
                                                metahtml(_items, _li_node)  # TODO
                                elif hasattr(value, '_repr_html_'):
                                    text(value._repr_html_(), escape=False)
                                #if isinstance(value, dominator.tag)
                                elif hasattr(value, 'render'):
                                    text(value.render(), escape=False)
                                #TODO: markupsafe __html__ ("?)
                                else:
                                    raise Exception((type(value), value))
                return cur_node
            doc_ = metahtml(self.meta.items(), doc)
        return doc.render()
    
    def _repr_html_(self):
        obj = self.obj

        if hasattr(obj, '_repr_html_'):
            obj_html = obj._repr_html_()
        else:
            obj_html = str(obj)
        print(type(obj_html))
        meta_html = self.meta_to_html()
        print(type(meta_html))
        return u'\n'.join((obj_html, self.meta_to_html()))


ns = OrderedDict()
schema = ns['schema'] = {
    "url": "http://schema.org/url",
    "author": "http://schema.org/author",
    "givenName": "http://schema.org/givenName",
    "Person": "http://schema.org/Person"
}
rdf = ns['rdf'] = {"a": "rdf:type"}
ReprHTML(
    Meta(this.render(),
        meta=OrderedDict([
         (schema['url'], "http://localhost:8888/notebooks/nb/...ipynb"),
         (schema['author'], [
             OrderedDict([
                 (rdf["a"], schema['Person']),
                 (schema['givenName'], "awesome")
              ]),
         ]),
        ])
    )
)

# TODO
# RreprJSONLD() RDFa

<type 'unicode'>
<type 'unicode'>


In [70]:
"""
Environment(requirementstxt='../requirements-2.txt')
Include('../README.rst')
Chart(plot, data, meta={})
"""

def _repr_html_():
  # emit and collect
  # ...
  globals()['celldata'][cell_n] = value
  return value

In [72]:
"""
this_notebook = TODO_GET_HERE()
base URI
 XHTML
 HTML5
https://schema.org/mainEntityOfPage
<>
"""

'\nthis_notebook = TODO_GET_HERE()\nbase URI\n XHTML\n HTML5\nhttps://schema.org/mainEntityOfPage\n<>\n'